<a href="https://colab.research.google.com/github/pdragojevic/Predicting-NBA-Shots/blob/main/predicting_nba_shots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


# Učitavanje podataka

In [2]:
# load data
df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/UZOP/project/shot_logs.csv")
# tranform data column names to lowercase
df.columns = df.columns.str.lower()

# Početni pregled podataka

In [6]:
df.head()

,game_id,matchup,location,w,final_margin,shot_number,period,game_clock,shot_clock,dribbles,...,shot_dist,pts_type,shot_result,closest_defender,closest_defender_player_id,close_def_dist,fgm,pts,player_name,player_id
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,...,7.7,2,made,"Anderson, Alan",101187,1.3,1,2,brian roberts,203148
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,...,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,0,0,brian roberts,203148
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,NaN,3,...,10.1,2,missed,"Bogdanovic, Bojan",202711,0.9,0,0,brian roberts,203148
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,...,17.2,2,missed,"Brown, Markel",203900,3.4,0,0,brian roberts,203148
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,10:34,10.9,2,...,3.7,2,missed,"Young, Thaddeus",201152,1.1,0,0,brian roberts,203148


In [4]:
df.columns.values

array(['game_id', 'matchup', 'location', 'w', 'final_margin',
       'shot_number', 'period', 'game_clock', 'shot_clock', 'dribbles',
       'touch_time', 'shot_dist', 'pts_type', 'shot_result',
       'closest_defender', 'closest_defender_player_id', 'close_def_dist',
       'fgm', 'pts', 'player_name', 'player_id'], dtype=object)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128069 entries, 0 to 128068
Data columns (total 21 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   game_id                     128069 non-null  int64  
 1   matchup                     128069 non-null  object 
 2   location                    128069 non-null  object 
 3   w                           128069 non-null  object 
 4   final_margin                128069 non-null  int64  
 5   shot_number                 128069 non-null  int64  
 6   period                      128069 non-null  int64  
 7   game_clock                  128069 non-null  object 
 8   shot_clock                  122502 non-null  float64
 9   dribbles                    128069 non-null  int64  
 10  touch_time                  128069 non-null  float64
 11  shot_dist                   128069 non-null  float64
 12  pts_type                    128069 non-null  int64  
 13  shot_result   

# Pretprocesiranje podataka

Kreiranje kopije podataka nad kojom ćemo raditi pretprocesiranje.

In [49]:
new_df = df.copy()

Zamjena kategoričkih značajki sa vrijednostima 0 i 1, pretvorba game_clock u sekunde

In [50]:
# home/away game => 1/0
new_df.location = new_df.location.map({"H":1,"A":0})
# win/lose => 1/0
new_df.w = new_df.w.map({"W":1,"L":0})
# 2 pointer/ 3 pointer => 0/1
new_df.pts_type = new_df.pts_type.map({2:0,3:1})
new_df.shot_result = new_df.shot_result.map({"made":1,"missed":0})

new_df = new_df.rename(columns={"location":"home_away","pts_type":"3pts_shot"})

new_df.game_clock = new_df.game_clock.apply(lambda x: int(x.split(":")[0])*60 + int(x.split(":")[1]))

## Inženjerstvo značajki

In [51]:
new_df.corr()

,game_id,home_away,w,final_margin,shot_number,period,game_clock,shot_clock,dribbles,touch_time,shot_dist,3pts_shot,shot_result,closest_defender_player_id,close_def_dist,fgm,pts,player_id
game_id,1.000000,-0.005014,-0.001730,0.001726,0.008129,-0.008921,0.002803,0.014530,0.002014,-0.000735,-0.000187,0.003418,-0.008761,0.028676,0.006739,-0.008761,-0.008464,0.027937
home_away,-0.005014,1.000000,0.115854,0.155075,-0.003267,0.003064,0.001685,0.008083,-0.014873,-0.012914,-0.001589,0.005012,0.008093,-0.007922,0.003579,0.008093,0.009179,0.011302
w,-0.001730,0.115854,1.000000,0.809887,-0.004633,-0.009667,0.013833,-0.021923,0.013406,0.013839,0.004876,0.013865,0.050239,0.050711,0.028071,0.050239,0.053999,-0.053149
final_margin,0.001726,0.155075,0.809887,1.000000,0.000836,-0.003078,0.005975,-0.009276,0.011739,0.010788,0.008876,0.024134,0.058762,0.057165,0.033064,0.058762,0.064784,-0.050825
shot_number,0.008129,-0.003267,-0.004633,0.000836,1.000000,0.654932,-0.237956,-0.032683,0.141100,0.144071,0.011541,0.003054,-0.008351,0.013257,-0.037769,-0.008351,-0.009666,-0.006592
period,-0.008921,0.003064,-0.009667,-0.003078,0.654932,1.000000,-0.027042,-0.051796,0.055301,0.046089,0.030080,0.048915,-0.014082,0.011462,-0.010204,-0.014082,-0.008501,0.004790
game_clock,0.002803,0.001685,0.013833,0.005975,-0.237956,-0.027042,1.000000,0.021242,-0.116189,-0.103308,-0.021729,-0.048474,0.011476,-0.006752,0.007455,0.011476,0.006650,-0.006176
shot_clock,0.014530,0.008083,-0.021923,-0.009276,-0.032683,-0.051796,0.021242,1.000000,-0.082771,-0.147688,-0.173026,-0.032809,0.096855,-0.003289,0.026558,0.096855,0.084173,0.038315
dribbles,0.002014,-0.014873,0.013406,0.011739,0.141100,0.055301,-0.116189,-0.082771,1.000000,0.914311,-0.082540,-0.169016,-0.034390,0.016915,-0.153674,-0.034390,-0.055689,0.023655
touch_time,-0.000735,-0.012914,0.013839,0.010788,0.144071,0.046089,-0.103308,-0.147688,0.914311,1.000000,-0.085165,-0.181007,-0.045299,0.016225,-0.164737,-0.045299,-0.066736,0.001797


Izbacujemo fgm i pts zato što savršeno ili skoro savršeno predviđaju da li je koš zabijen. Od ostalih značajki također izbacujemo game_id, matchup, closest_defender, player_name.

In [52]:
new_df = new_df.drop(columns =['game_id',
                               'matchup',
                               'closest_defender',
                               'player_name',
                               'fgm',
                               'pts'])

## Provjera nedostajućih vrijednosti.

In [80]:
new_df.isna().sum()

home_away                     0
w                             0
final_margin                  0
shot_number                   0
period                        0
game_clock                    0
shot_clock                    0
dribbles                      0
touch_time                    0
shot_dist                     0
3pts_shot                     0
shot_result                   0
closest_defender_player_id    0
close_def_dist                0
player_id                     0
dtype: int64

In [62]:
new_df[new_df.shot_clock.isna()].head()

,home_away,w,final_margin,shot_number,period,game_clock,shot_clock,dribbles,touch_time,shot_dist,3pts_shot,shot_result,closest_defender_player_id,close_def_dist,player_id
2,0,1,24,3,1,0,NaN,3,2.7,10.1,0,0,202711,0.9,203148
24,0,1,12,6,2,4,NaN,0,0.7,25.2,1,0,202710,3.6,203148
54,1,0,-1,14,4,1,NaN,5,3.7,34.4,1,0,201588,3.6,203148
76,1,1,7,10,3,1,NaN,2,3.6,13.1,0,1,2446,2.5,203148
129,1,1,25,7,2,2,NaN,5,5.0,12.0,0,0,2590,4.0,203148


Zaključujem da se radi o završnim trenucima četvrtine, ostalo je manje od 24 sekunde, stoga u tim slučajevima shot_clock možemo zamijeniti s game_clock. Slučaj u kojem je game_clock veći od 24 možemo smatrati neisrpavnim unosom i staviti prosječno vrijeme za shot_clock.

In [75]:
print(len(new_df[(new_df.shot_clock.isna()) & (new_df.game_clock<24)]))
print(len(new_df[(new_df.shot_clock.isna()) & (new_df.game_clock>=24)]))

0
0


In [74]:
new_df.loc[(new_df.shot_clock.isna()) & (new_df.game_clock<24), 'shot_clock'] = new_df.loc[:,'game_clock']
new_df.loc[new_df.shot_clock.isna(), 'shot_clock'] = new_df.loc[:,'shot_clock'].mean()

## Provjera stršećih podataka

In [76]:
new_df.describe()

,home_away,w,final_margin,shot_number,period,game_clock,shot_clock,dribbles,touch_time,shot_dist,3pts_shot,shot_result,closest_defender_player_id,close_def_dist,player_id
count,124711.000000,124711.000000,124711.000000,124711.000000,124711.000000,124711.000000,124711.000000,124711.000000,124711.000000,124711.000000,124711.000000,124711.000000,124711.000000,124711.000000,124711.000000
mean,0.499170,0.504695,0.213462,6.501856,2.469189,351.881101,12.101045,2.077716,2.846596,13.745229,0.269279,0.451885,159110.685377,4.145443,157107.137109
std,0.500001,0.499980,13.235574,4.715032,1.140324,207.544406,5.754747,3.508217,2.991336,8.848028,0.443587,0.497682,78721.980843,2.742992,79421.362932
min,0.000000,0.000000,-53.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,708.000000,0.000000,708.000000
25%,0.000000,0.000000,-8.000000,3.000000,1.000000,173.000000,8.000000,0.000000,0.900000,4.900000,0.000000,0.000000,200745.000000,2.300000,101162.000000
50%,0.000000,1.000000,1.000000,5.000000,2.000000,353.000000,12.100000,1.000000,1.700000,14.100000,0.000000,0.000000,201949.000000,3.700000,201939.000000
75%,1.000000,1.000000,9.000000,9.000000,3.000000,532.000000,16.200000,3.000000,3.800000,22.600000,1.000000,1.000000,203079.000000,5.300000,202704.000000
max,1.000000,1.000000,53.000000,38.000000,7.000000,720.000000,24.000000,32.000000,24.900000,47.200000,1.000000,1.000000,530027.000000,53.200000,204060.000000


touch_time ne bi smio biti negativan, takvih zapisa je 312, pošto ih ima tako malo možemo ih izbaciti.

In [77]:
len(new_df[new_df.touch_time<0])

0

In [78]:
new_df = new_df[new_df['touch_time']>0]

In [79]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124711 entries, 0 to 128068
Data columns (total 15 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   home_away                   124711 non-null  int64  
 1   w                           124711 non-null  int64  
 2   final_margin                124711 non-null  int64  
 3   shot_number                 124711 non-null  int64  
 4   period                      124711 non-null  int64  
 5   game_clock                  124711 non-null  int64  
 6   shot_clock                  124711 non-null  float64
 7   dribbles                    124711 non-null  int64  
 8   touch_time                  124711 non-null  float64
 9   shot_dist                   124711 non-null  float64
 10  3pts_shot                   124711 non-null  int64  
 11  shot_result                 124711 non-null  int64  
 12  closest_defender_player_id  124711 non-null  int64  
 13  close_def_dist